In [6]:
import tensorflow as tf
import keras_nlp
from transformer import TransformerModel

In [7]:
input_sequence_length = 403
output_sequence_length = 402
embed_dim = 128
dense_dim = 512
num_heads = 8
num_layers = 2
dropout_rate = 0.3

model_path = '../models/100_l2h8d128/model.ckpt'

input_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary = 'C:/Users/Ivan/Desktop/CODE/vocab/spaces_vocab',
    sequence_length = input_sequence_length,
    oov_token = 'UNK'
)
input_vocab_size = input_tokenizer.vocabulary_size()

output_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary = 'C:/Users/Ivan/Desktop/CODE/vocab/pairs_vocab',
    sequence_length = output_sequence_length + 1,
    oov_token = 'UNK'
)
output_vocab_size = output_tokenizer.vocabulary_size()

In [8]:
model = TransformerModel(input_vocab_size,
                         output_vocab_size,
                         input_sequence_length,
                         output_sequence_length,
                         embed_dim,
                         dense_dim,
                         num_heads,
                         num_layers,
                         dropout_rate)

model.load_weights(model_path)

In [ ]:
test_data = 'alignments_200_gap5_test_cleaned'

file = open(test_data, 'r')
lines = file.readlines()
file.close()
for line in lines:
    print('NEW LINE')
    unaligned, aligned, score = line.strip('\n').split('\t')
    
    seq1 = unaligned.split('|')[0].strip(' ').replace(' ', '')
    seq2 = unaligned.split('|')[1].strip(' ').replace(' ', '')
    
    print(seq1)
    print(seq2)
    
    unaligned = 'START ' + unaligned + ' END'    
    encoder_input = input_tokenizer([unaligned])
    decoder_input = 'START'
    for i in range(output_sequence_length):
        tokenized_decoder_input = output_tokenizer([decoder_input])[:, :-1]
        predictions = model((encoder_input, tokenized_decoder_input))
        
        new_token_index = tf.argmax(predictions[0, i, :])
        
        new_token = output_tokenizer.vocabulary[new_token_index]
        decoder_input = decoder_input + ' ' + new_token
        
        if(new_token == 'END'):
            break    
    print(decoder_input)
    print('\n')

In [11]:
seq1 = 'MSLFEQLGGQAAVQAVTAQFYANIQADATVATFFNGIDMPNQTNKTAAFLCAALGGPNAWTGRNLKEVHANMGVSNAQFTTVIGHLRSALTGAGVAAA'
seq2 = 'RRGDMTEQGKKLMQMIGAAVGKLNDLDNLVPVLQGLAVRHDGYGVKESHYQTVGAALLKTLGQGLGDRFTSETRDAWATVYGVMANVMITASRAT'

unaligned = ' '.join(seq1) + ' | ' + ' '.join(seq2)
print(unaligned)
    
unaligned = 'START ' + unaligned + ' END'    
encoder_input = input_tokenizer([unaligned])
decoder_input = 'START'
for i in range(output_sequence_length):
    tokenized_decoder_input = output_tokenizer([decoder_input])[:, :-1]
    enc_out, padd = model.encoder(encoder_input)
    print(enc_out.shape)
    predictions = model((encoder_input, tokenized_decoder_input))
    
    print(predictions.shape)
        
    new_token_index = tf.argmax(predictions[0, i, :])
        
    new_token = output_tokenizer.vocabulary[new_token_index]
    decoder_input = decoder_input + ' ' + new_token
    
    print(decoder_input)
        
    if(new_token == 'END'):
        break    
print(decoder_input)
print('\n')

M S L F E Q L G G Q A A V Q A V T A Q F Y A N I Q A D A T V A T F F N G I D M P N Q T N K T A A F L C A A L G G P N A W T G R N L K E V H A N M G V S N A Q F T T V I G H L R S A L T G A G V A A A | R R G D M T E Q G K K L M Q M I G A A V G K L N D L D N L V P V L Q G L A V R H D G Y G V K E S H Y Q T V G A A L L K T L G Q G L G D R F T S E T R D A W A T V Y G V M A N V M I T A S R A T
(1, 403, 128)
(1, 402, 445)
START Mx
(1, 403, 128)
(1, 402, 445)
START Mx Sx
(1, 403, 128)
(1, 402, 445)
START Mx Sx Lx
(1, 403, 128)
(1, 402, 445)
START Mx Sx Lx Fx
(1, 403, 128)
(1, 402, 445)
START Mx Sx Lx Fx Ex
(1, 403, 128)
(1, 402, 445)
START Mx Sx Lx Fx Ex Qx
(1, 403, 128)
(1, 402, 445)
START Mx Sx Lx Fx Ex Qx Lx
(1, 403, 128)
(1, 402, 445)
START Mx Sx Lx Fx Ex Qx Lx Gx
(1, 403, 128)
(1, 402, 445)
START Mx Sx Lx Fx Ex Qx Lx Gx Gx
(1, 403, 128)
(1, 402, 445)
START Mx Sx Lx Fx Ex Qx Lx Gx Gx Qx
(1, 403, 128)
(1, 402, 445)
START Mx Sx Lx Fx Ex Qx Lx Gx Gx Qx Ax
(1, 403, 128)
(1, 402, 445)
START Mx Sx 

KeyboardInterrupt: 